In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# URLs for datasets
datasets = {
    "SO2TONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/SO2TONS_dataset.csv",
    "NOXTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/NOXTONS_dataset.csv",
    "COTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/COTONS_dataset.csv"
}

# Define the peak season months (May through August)
peak_season_months = [5, 6, 7, 8]

# Define lakes (sources)
sources = ["LAKE-1", "LAKE-2", "LAKE-3", "LAKE-4"]

# Define the specific day for prediction
specific_date = pd.Timestamp("2022-07-15")

# Initialize a dictionary to store models, predictions, and inputs for verification
models = {}
predictions = {}

# Loop through each dataset (SO2TONS, NOXTONS, COTONS)
for parameter, url in datasets.items():
    # Load the dataset
    data = pd.read_csv(url)

    # Convert the 'date' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Filter for peak season
    data = data[data['date'].dt.month.isin(peak_season_months)]

    # Separate data by source
    for source in sources:
        source_data = data[data['Source'] == source]

        # Check if the source data has enough rows
        if source_data.empty or len(source_data) < 10:
            print(f"Not enough data for {parameter} at {source}. Skipping...")
            continue

        # Define predictors (e.g., weather features)
        predictors = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']
        target = 'Emissions_Load'

        # Drop rows with missing values
        source_data = source_data.dropna(subset=predictors + [target])

        # Split the data into features (X) and target (y)
        X = source_data[predictors]
        y = source_data[target]

        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        # Define a KNN model
        model = KNeighborsRegressor(n_neighbors=5, weights='distance')

        # Train the model
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print(f"Model for {parameter} at {source}:")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  R²: {r2:.4f}")

        # Save the model and scaler
        models[(parameter, source)] = (model, scaler)

        # Check if the specific date exists in the source data
        day_data = source_data[source_data['date'] == specific_date]
        if not day_data.empty:
            # Extract feature values for the specific day
            specific_features = scaler.transform(day_data[predictors])
            specific_actual = day_data[target].iloc[0]

            # Predict emissions/load for the specific day
            specific_prediction = model.predict(specific_features)[0]

            # Save the prediction and actual value for verification
            predictions[(parameter, source)] = {
                "features": day_data[predictors].iloc[0],
                "actual": specific_actual,
                "predicted": specific_prediction
            }

# Display all predictions at the end
print("\nFinal Predictions:")
for key, value in predictions.items():
    parameter, source = key
    print(f"{parameter} at {source}:")
    print(f"  Features: {value['features'].to_dict()}")
    print(f"  Actual Emissions_Load: {value['actual']:.4f}")
    print(f"  Predicted Emissions_Load: {value['predicted']:.4f}")
    print()


Model for SO2TONS at LAKE-1:
  RMSE: 0.0000
  R²: 0.1849
Model for SO2TONS at LAKE-2:
  RMSE: 0.0000
  R²: -0.0404
Model for SO2TONS at LAKE-3:
  RMSE: 0.0000
  R²: 0.1527
Model for SO2TONS at LAKE-4:
  RMSE: 0.0000
  R²: 0.2443
Model for NOXTONS at LAKE-1:
  RMSE: 0.0003
  R²: -0.0447
Model for NOXTONS at LAKE-2:
  RMSE: 0.0003
  R²: -0.3478
Model for NOXTONS at LAKE-3:
  RMSE: 0.0002
  R²: 0.1516
Model for NOXTONS at LAKE-4:
  RMSE: 0.0002
  R²: 0.1956
Model for COTONS at LAKE-1:
  RMSE: 0.0001
  R²: -0.1136
Model for COTONS at LAKE-2:
  RMSE: 0.0001
  R²: -0.4835
Model for COTONS at LAKE-3:
  RMSE: 0.0001
  R²: 0.1663
Model for COTONS at LAKE-4:
  RMSE: 0.0001
  R²: 0.2429

Final Predictions:
SO2TONS at LAKE-1:
  Features: {'tavg': 31.7, 'tmin': 23.3, 'tmax': 38.9, 'prcp': 0.0, 'snow': 0.0, 'wdir': 87.0, 'wspd': 11.2, 'pres': 1011.3}
  Actual Emissions_Load: 0.0000
  Predicted Emissions_Load: 0.0000

SO2TONS at LAKE-2:
  Features: {'tavg': 31.7, 'tmin': 23.3, 'tmax': 38.9, 'prcp': 0